# Homework 5 Part 2 (Collaborative tasks)
***
## Members:
- Hans Lehndorff
- Isaac Johnson
- Jesse DeBolt

## Setup

In [49]:
import pandas as pd
import numpy as np

# Set working directory (modify as needed)
%cd -q "C:\Users\jesse\Documents\Schooling\Willamette-MSDS\1-DATA599-02_Python\Final"

### Importing data

In [27]:
# Import the dataset
cdc = pd.read_csv('data/CDC_for_python.csv')

In [28]:
# View number of rows and columns
cdc.shape

(3226, 59)

***

## Data cleaning

### Removing US territories 

In [29]:
cdc = cdc[~cdc['display_name'].str.contains(r'\(AS\)|\(GU\)|\(MP\)|\(PR\)|\(County Equivalent\)')]

In [31]:
# View number of rows to confirm removal
cdc.shape

(3142, 59)

### Separate county and state

In [79]:
# Separate the 'display_name' column
cdc[['county', 'state']] = cdc['display_name'].str.extract(r'\"(.+), \((.+)\)\"', expand=True)

# Remove the original column
cdc = cdc.drop(['display_name'], axis=1)

In [80]:
cdc.head()

,fips,Age65Plus,AIAN,AIANmen,AIANwomen,ANHPI,ANHPImen,ANHPIwomen,Black,BlackMen,...,EdLessColl,SNAPrecipients,MedHomeValue,MedHouseIncome,Poverty,Unemploy,UrbanRural,county,state,pcp
0,34025,17.7,781.0,508.0,273.0,26636.0,12508.0,14128.0,33615.0,15605.0,...,52.7,4.5,435000.0,104000.0,5.9,5.5,LargeFringe_Urban,Monmouth,NJ,0.8
1,34019,18.6,121.0,74.0,47.0,4082.0,1924.0,2158.0,2890.0,1882.0,...,46.0,1.9,419000.0,114000.0,4.1,4.6,LargeFringe_Urban,Hunterdon,NJ,0.8
2,34017,12.0,2619.0,1539.0,1080.0,88314.0,44162.0,44152.0,63097.0,29672.0,...,56.0,13.1,401000.0,77000.0,13.1,6.8,Large_Urban,Hudson,NJ,1.8
3,34023,15.1,1399.0,795.0,604.0,152352.0,76218.0,76134.0,66271.0,31215.0,...,55.6,6.0,351000.0,96000.0,7.4,5.7,LargeFringe_Urban,Middlesex,NJ,1.0
4,34001,18.1,872.0,498.0,374.0,16216.0,7733.0,8483.0,29354.0,13401.0,...,71.2,13.3,217000.0,61000.0,13.8,9.5,MediumSmall_Urban,Atlantic,NJ,1.2


### Relabeling Urban/Rural codes

In [41]:
#change rural/urban
# 1 = Large central metro -> Large_Urban
# 2 = Large fringe metro -> LargeFringe_Urban
# 3 = Medium/small metro -> MediumSmall_Urban
# 4 = Nonmetro -> Rural

# Replace values in 'UrbanRural' column
cdc['UrbanRural'] = cdc['UrbanRural'].replace({1: 'Large_Urban', 2: 'LargeFringe_Urban', 3: 'MediumSmall_Urban', 4: 'Rural'})

# Replace -1 with NaN
cdc = cdc.replace(-1, np.nan)

### Review header names, check for missing rural/urban values

In [52]:
# Display column names
print(cdc.columns)

# Show unique values in 'UrbanRural'
print(cdc['UrbanRural'].unique())

# Create a subset where 'UrbanRural' is NaN, an empty string, or 'NA'
rural_query = cdc[cdc['UrbanRural'].isin([pd.np.nan, '', 'NA'])]

# Display the first 20 rows of this subset
rural_query.head(20)

Index(['fips', 'display_name', 'Age65Plus', 'AIAN', 'AIANmen', 'AIANwomen',
       'ANHPI', 'ANHPImen', 'ANHPIwomen', 'Black', 'BlackMen', 'BlackWomen',
       'Hispanic', 'HispanicMen', 'HispanicWomen', 'OtherRace', 'OtherRaceMen',
       'OtherRaceWomen', 'PopAllGenders', 'PopMen', 'PopWomen', 'pop',
       'TwoPlus', 'TwoPlusMen', 'TwoPlusWomen', 'White', 'WhiteMen',
       'WhiteWomen', 'bpmUse', 'CholScreen', 'CholMedNonAdhear',
       'CholMedElegible', 'cruParticipate', 'Hospitals', 'HospCIC', 'HospCR',
       'HospED', 'Pharmacies', 'HealthIns', 'CardioPhys', 'PrimaryCarePhys',
       'CHD', 'HighBP', 'Stroke', 'Diabetes', 'HighChol', 'Obesity',
       'PhysInactivity', 'Smoker', 'AirQuality', 'Parks', 'Broadband',
       'EdLessColl', 'SNAPrecipients', 'MedHomeValue', 'MedHouseIncome',
       'Poverty', 'Unemploy', 'UrbanRural', 'county', 'state'],
      dtype='object')
['Rural' 'MediumSmall_Urban' nan 'Large_Urban' 'LargeFringe_Urban']


C:\Users\jesse\AppData\Local\Temp\ipykernel_8840\1803092657.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  rural_query = cdc[cdc['UrbanRural'].isin([pd.np.nan, '', 'NA'])]


,fips,display_name,Age65Plus,AIAN,AIANmen,AIANwomen,ANHPI,ANHPImen,ANHPIwomen,Black,...,Broadband,EdLessColl,SNAPrecipients,MedHomeValue,MedHouseIncome,Poverty,Unemploy,UrbanRural,county,state
25,2158,"""Kusilvak, (AK)""",5.8,4431.0,2361.0,2070.0,17.0,13.0,4.0,39.0,...,33.9,96.7,48.7,73000.0,38000.0,27.9,19.9,NaN,Kusilvak,AK


### Inserting Rural/Urban for missing value

In [54]:
# Find and insert where county is 'Kusilvak' based on Wikipedia data.
cdc.loc[cdc['county'].str.contains('Kusilvak', na=False), 'UrbanRural'] = "Rural"

# Display unique values in the 'UrbanRural' column
print(cdc['UrbanRural'].unique())

# Double check for any NAs in Rural/Urban
rural_query = cdc[cdc['UrbanRural'].isin([None, "", "NA"])].head(20)
rural_query

['Rural' 'MediumSmall_Urban' 'Large_Urban' 'LargeFringe_Urban']


,fips,display_name,Age65Plus,AIAN,AIANmen,AIANwomen,ANHPI,ANHPImen,ANHPIwomen,Black,...,Broadband,EdLessColl,SNAPrecipients,MedHomeValue,MedHouseIncome,Poverty,Unemploy,UrbanRural,county,state


### Convert fips to string

In [62]:
cdc['fips'] = cdc['fips'].astype(str)

### Inserting Parks missing value

In [55]:
cdc.loc[cdc['county'].str.contains('Kusilvak', na=False), 'Parks'] = 66

### Checking for missing values

In [58]:
# Total number of missing values in dataset
total_na = cdc.isnull().sum().sum()
print(total_na)

# Total number of missing values in each column
column_na = cdc.isnull().sum()
print(column_na)

# Total number of missing values in each row
cdc['count_na'] = cdc.isnull().sum(axis=1)

# Sort by 'count_na'
cdc = cdc.sort_values(by='count_na', ascending=False)

3286
fips            0
display_name    0
Age65Plus       0
AIAN            0
AIANmen         0
               ..
Poverty         0
Unemploy        0
UrbanRural      0
county          0
state           0
Length: 61, dtype: int64


### Remove counties with minimal data

In [59]:
# Remove those that have more than 8 NAs in that row
cdc = cdc[cdc['count_na'] <= 8]

# Remove 'count_na' column
cdc = cdc.drop('count_na', axis=1)

### Inserting values for missing data for NJ
Note: These values were obtained from 500 Cities & Places data (https://chronicdata.cdc.gov/browse)

In [61]:
# bpmUse
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'bpmUse'] = 71.71

# CholScreen
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'CholScreen'] = 79.43

# HighBP
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'HighBP'] = 33.7

# Diabetes
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'Diabetes'] = 17.4

# HighChol
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'HighChol'] = 32.41

# Obesity
cdc.loc[cdc['state'].str.contains('NJ', na=False), 'Obesity'] = 33.59

### Inserting values for missing values in Median Home Value
Note: these values were obtained from city-data.com (https://www.city-data.com)

In [64]:
cdc.loc[cdc['fips'].str.contains('48261', na=False), 'MedHomeValue'] = 42550
cdc.loc[cdc['fips'].str.contains('48301', na=False), 'MedHomeValue'] = 38143
cdc.loc[cdc['fips'].str.contains('46017', na=False), 'MedHomeValue'] = 101393
cdc.loc[cdc['fips'].str.contains('46095', na=False), 'MedHomeValue'] = 60537

### Inserting missing values for PCP and Cardio Phys

In [65]:
# Read in dataset with values for merging
pcp_cardio_count = pd.read_csv("data/pcp_cardio_count.csv")

# Convert the 'COUNTY' column to string
pcp_cardio_count['COUNTY'] = pcp_cardio_count['COUNTY'].astype(str)

# Joining count data frame to the cdc data frame
cdc = cdc.merge(pcp_cardio_count, left_on='fips', right_on='COUNTY', how='left')

# Use fillna to replace NAs in PCP and CardioPhys
cdc['pcp'] = cdc['pcp'].fillna(cdc['PrimaryCarePhys'])
cdc['CardioPhys'] = cdc['CardioPhys'].fillna(cdc['cardio'])

# Remove the temporary columns
cdc = cdc.drop(['PrimaryCarePhys', 'cardio', 'COUNTY'], axis=1)

### Check entire data frame for remaining missing or NA values

In [76]:
# Check DataFrame for missing or NA values, but only display columns with missing values
missing_values = cdc.isnull().sum()
missing_values = missing_values[missing_values != 0]
print(missing_values)

CholMedNonAdhear      80
CholMedElegible      110
cruParticipate       742
CardioPhys          1493
PhysInactivity        21
AirQuality            24
pcp                   25
dtype: int64


In [78]:
cdc

,fips,display_name,Age65Plus,AIAN,AIANmen,AIANwomen,ANHPI,ANHPImen,ANHPIwomen,Black,...,EdLessColl,SNAPrecipients,MedHomeValue,MedHouseIncome,Poverty,Unemploy,UrbanRural,county,state,pcp
0,34025,"""Monmouth, (NJ)""",17.7,781.0,508.0,273.0,26636.0,12508.0,14128.0,33615.0,...,52.7,4.5,435000.0,104000.0,5.9,5.5,LargeFringe_Urban,Monmouth,NJ,0.8
1,34019,"""Hunterdon, (NJ)""",18.6,121.0,74.0,47.0,4082.0,1924.0,2158.0,2890.0,...,46.0,1.9,419000.0,114000.0,4.1,4.6,LargeFringe_Urban,Hunterdon,NJ,0.8
2,34017,"""Hudson, (NJ)""",12.0,2619.0,1539.0,1080.0,88314.0,44162.0,44152.0,63097.0,...,56.0,13.1,401000.0,77000.0,13.1,6.8,Large_Urban,Hudson,NJ,1.8
3,34023,"""Middlesex, (NJ)""",15.1,1399.0,795.0,604.0,152352.0,76218.0,76134.0,66271.0,...,55.6,6.0,351000.0,96000.0,7.4,5.7,LargeFringe_Urban,Middlesex,NJ,1.0
4,34001,"""Atlantic, (NJ)""",18.1,872.0,498.0,374.0,16216.0,7733.0,8483.0,29354.0,...,71.2,13.3,217000.0,61000.0,13.8,9.5,MediumSmall_Urban,Atlantic,NJ,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3135,51083,"""Halifax, (VA)""",24.2,58.0,1.0,57.0,139.0,52.0,87.0,9473.0,...,83.9,16.7,119000.0,43000.0,17.8,4.8,Rural,Halifax,VA,45.0
3136,51085,"""Hanover, (VA)""",17.8,275.0,48.0,227.0,1411.0,671.0,740.0,7732.0,...,59.7,4.0,289000.0,98000.0,5.0,3.0,LargeFringe_Urban,Hanover,VA,126.0
3137,51087,"""Henrico, (VA)""",15.5,401.0,253.0,148.0,22234.0,11124.0,11110.0,74905.0,...,55.7,8.8,254000.0,80000.0,8.3,4.0,LargeFringe_Urban,Henrico,VA,489.0
3138,36031,"""Essex, (NY)""",23.5,72.0,24.0,48.0,118.0,43.0,75.0,1139.0,...,70.2,8.6,160000.0,57000.0,10.2,4.7,Rural,Essex,NY,60.0


### Impute for missing values

In [84]:
from sklearn.impute import SimpleImputer

# Specify the columns you want to impute
columns_to_impute = ['CholMedNonAdhear', 'CholMedElegible', 'cruParticipate', 'CardioPhys', 'PhysInactivity', 'AirQuality', 'pcp']

# Subset the DataFrame to only these columns
subset_cdc = cdc[columns_to_impute]

# Create an imputer object
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to the data and transform the data
imputed_data = imputer.fit_transform(subset_cdc)

# Convert the result back to a DataFrame
imputed_data = pd.DataFrame(imputed_data, columns=subset_cdc.columns)

# Replace the original columns in the DataFrame with the imputed data
cdc[columns_to_impute] = imputed_data


### Check entire data frame for remaining missing or NA values

In [85]:
# Check DataFrame for missing or NA values, but only display columns with missing values
missing_values = cdc.isnull().sum()
missing_values = missing_values[missing_values != 0]
print(missing_values)

Series([], dtype: int64)


### Export data fram to csv file

In [87]:
cdc.to_csv('data/CDC_python_clean.csv', index=False)